## ch12 Hyperopt
- https://github.com/mattharrison/effective_xgboost_book/blob/main/xgbcode.ipynb

<div style="text-align: right"> <b>Author : Kwang Myung Yu</b></div>
<div style="text-align: right"> Initial upload: 2023.8.1</div>
<div style="text-align: right"> Last update: 2023.8.1</div>

In [1]:
import os
import sys
import time
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
from scipy import stats
import warnings; warnings.filterwarnings('ignore')
#plt.style.use('ggplot')
plt.style.use('seaborn-whitegrid')
%matplotlib inline

새롭게 라이브러리를 로드 하자.

In [2]:
import dtreeviz
from feature_engine import encoding, imputation
import numpy as np
import pandas as pd
from sklearn import base, compose, datasets, ensemble, \
    metrics, model_selection, pipeline, preprocessing, tree
import scikitplot
import xgboost as xgb
import yellowbrick.model_selection as ms
from yellowbrick import classifier

import urllib
import zipfile

import xg_helpers as xhelp
from xg_helpers import my_dot_export

In [3]:
url = 'https://github.com/mattharrison/datasets/raw/master/data/'\
'kaggle-survey-2018.zip'
fname = 'kaggle-survey-2018.zip'
member_name = 'multipleChoiceResponses.csv'

In [4]:
raw = xhelp.extract_zip(url, fname, member_name)
## Create raw X and raw y
kag_X, kag_y = xhelp.get_rawX_y(raw, 'Q6')

In [5]:
## Split data
kag_X_train, kag_X_test, kag_y_train, kag_y_test = \
model_selection.train_test_split(
kag_X, kag_y, test_size=.3, random_state=42, stratify=kag_y)

In [6]:
## Transform X with pipeline
X_train = xhelp.kag_pl.fit_transform(kag_X_train)
X_test = xhelp.kag_pl.transform(kag_X_test)
## Transform y with label encoder
label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(kag_y_train)
y_train = label_encoder.transform(kag_y_train)
y_test = label_encoder.transform(kag_y_test)
# Combined Data for cross validation/etc
X = pd.concat([X_train, X_test], axis='index')
y = pd.Series([*y_train, *y_test], index=X.index)

In [7]:
X.head()

,age,education,years_exp,compensation,python,r,sql,Q1_Male,Q1_Female,Q1_Prefer not to say,Q1_Prefer to self-describe,Q3_United States of America,Q3_India,Q3_China,major_cs,major_other,major_eng,major_stat
587,25,18.0,4.0,0,1,0,1,1,0,0,0,0,1,0,0,1,0,0
3065,22,16.0,1.0,10000,1,0,0,1,0,0,0,0,1,0,1,0,0,0
8435,22,18.0,1.0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0
3110,40,20.0,3.0,125000,1,0,1,0,1,0,0,1,0,0,0,1,0,0
16372,45,12.0,5.0,100000,1,0,1,1,0,0,0,1,0,0,0,1,0,0


In [8]:
y.head()

587      1
3065     0
8435     0
3110     0
16372    1
dtype: int64

Hyperopt uses Bayesian optimization to tune hyperparameters.  

This uses
a probabilistic model to select the next set of hyperparameters to try. If one value performs
better, it will try values around it and see if they boost the performance. If the values worsen
the model, then Hyperopt can ignore those values as future candidates. Hyperopt can also
tune various other machine-learning models, including random forests and neural networks.

### 12.1 Bayesian Optimization

### 12.2 Exhaustive Tuning with Hyperopt

파이썬에서는 함수를 다른 함수에 매개변수로 전달하거나 그 결과로 함수를 반환할 수 있습니다.  
Hyperopt 라이브러리를 사용하려면 많은 하이퍼파라미터 스페이스가 주어졌을 때 최적의 하이퍼파라미터 값을 찾으려고 시도하는 fmin 함수를 사용해야 합니다.

fmin 함수는 평가할 하이퍼파라미터의 딕셔너리를 받아들이고 딕셔너리를 반환하는 다른 함수를 전달할 것으로 예상합니다. 

hyperparameter_tuning 함수는 여기에 적합하지 않습니다. 

이 함수는 첫 번째 매개변수로 공간이라는 사전을 받지만 추가 매개변수가 있습니다. 람다 함수 형태의 클로저를 사용하여 hyperparameter_tuning 함수를 하이퍼파라미터 사전을 받아들이는 새로운 함수에 맞게 조정하겠습니다.

hyperparameter_tuning 함수는 하이퍼파라미터(공간), 훈련 데이터(X_train 및 y_train), 테스트 데이터(X_test 및 y_test), 조기 중지 라운드에 대한 선택적 값(early_stopping_rounds)의 딕셔너리(사전)를 받습니다. 일부 하이퍼파라미터는 정수여야 하므로 해당 키를 변환하고 평가할 하이퍼파라미터에 early_stopping_rounds를 추가합니다.

그런 다음 모델을 훈련하고 정확도 점수가 음수인 사전과 상태를 반환합니다.  
fmin은 점수를 최소화하려고 하고 우리 모델은 정확도를 평가하기 때문에 최소 정확도를 가진 모델을 원하지 않습니다. 그러나 약간의 트릭을 사용하여 음수 정확도를 최소화할 수 있습니다.

In [9]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.metrics import accuracy_score, roc_auc_score

from typing import Any, Dict, Union

In [10]:
def hyperparameter_tuning(space: Dict[str, Union[float, int]], 
                    X_train: pd.DataFrame, y_train: pd.Series, 
                    X_test: pd.DataFrame, y_test: pd.Series, 
                    early_stopping_rounds: int=50,
                    metric:callable=accuracy_score) -> Dict[str, Any]:
    """
    Perform hyperparameter tuning for an XGBoost classifier.

    This function takes a dictionary of hyperparameters, training 
    and test data, and an optional value for early stopping rounds, 
    and returns a dictionary with the loss and model resulting from 
    the tuning process. The model is trained using the training 
    data and evaluated on the test data. The loss is computed as 
    the negative of the accuracy score.

    Parameters
    ----------
    space : Dict[str, Union[float, int]]
        A dictionary of hyperparameters for the XGBoost classifier.
    X_train : pd.DataFrame
        The training data.
    y_train : pd.Series
        The training target.
    X_test : pd.DataFrame
        The test data.
    y_test : pd.Series
        The test target.
    early_stopping_rounds : int, optional
        The number of early stopping rounds to use. The default value 
        is 50.
    metric : callable
        Metric to maximize. Default is accuracy

    Returns
    -------
    Dict[str, Any]
        A dictionary with the loss and model resulting from the 
        tuning process. The loss is a float, and the model is an 
        XGBoost classifier.
    """
    int_vals = ['max_depth', 'reg_alpha']
    space = {k: (int(val) if k in int_vals else val)
             for k,val in space.items()}
    space['early_stopping_rounds'] = early_stopping_rounds
    model = xgb.XGBClassifier(**space)
    evaluation = [(X_train, y_train),
                  (X_test, y_test)]
    model.fit(X_train, y_train,
              eval_set=evaluation, 
              verbose=False)    

    pred = model.predict(X_test)
    score = metric(y_test, pred)
    return {'loss': -score, 'status': STATUS_OK, 'model': model}

최소화하려는 함수를 정의한 후에는 하이퍼파라미터를 검색할 공간을 정의해야 합니다.   

다음으로 하이퍼파라미터 튜닝 프로세스의 결과를 저장하기 위해 traial object를 생성합니다. 
fmin 함수는 옵션을 반복하여 최적의 하이퍼파라미터를 검색합니다.  
여기서는 tree-structured Parzen Estimator, 베이지안 최적화를 실행하는 tpe.suggest 알고리즘을 사용하도록 지시했습니다. 이 
 알고리즘은 expected improvement acquisition function을 사용하여 검색합니다. 
이 알고리즘은 노이즈가 있는 데이터에서 잘 작동합니다.   

또한 검색에 평가 시도를 2,000회로 제한하도록 지시했습니다.  
내 Macbook Pro(2022)에서 이 검색을 실행하는 데 약 30분이 걸렸습니다. 내 Thinkpad P1(2020)에서는 2시간이 넘게 걸립니다. 시간 제한 기간을 설정할 수 있습니다. 로 설정할 수도 있으며, 이 검색이 실행되면 최고의 손실 점수를 뱉어냅니다.

In [11]:
# options = {'max_depth': hp.quniform('max_depth', 1, 8, 1),  # tree
#     'min_child_weight': hp.loguniform('min_child_weight', -2, 3),
#     'subsample': hp.uniform('subsample', 0.5, 1),   # stochastic
#     'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
#     'reg_alpha': hp.uniform('reg_alpha', 0, 10),
#     'reg_lambda': hp.uniform('reg_lambda', 1, 10),
#     'gamma': hp.loguniform('gamma', -10, 10), # regularization
#     'learning_rate': hp.loguniform('learning_rate', -7, 0),  # boosting
#     'random_state': 42
# }

# trials = Trials()
# best = fmin(fn=lambda space: hyperparameter_tuning(space, X_train, y_train, 
#                                                    X_test, y_test),            
#     space=options,           
#     algo=tpe.suggest,            
#     max_evals=2_000,            
#     trials=trials,
#     #timeout=60*5 # 5 minutes
# )

best 변수에 결과가 포함된다.   
하이퍼파라미터를 출력해서 보관하자.

In [12]:
# best

주의 : max_depth는 정수이나 실수로 표시됨, 정수로 조정 필요

In [13]:
long_params = {'colsample_bytree': 0.7583844691864223,
 'gamma': 0.0006312831731257507,
 'learning_rate': 0.295815862267789,
 'max_depth': 7,
 'min_child_weight': 5.333178297222369,
 'reg_alpha': 5.486906586611888,
 'reg_lambda': 1.1524178084577712,
 'subsample': 0.9721881017707349}

In [14]:
xg_ex = xgb.XGBClassifier(**long_params, early_stopping_rounds=50,
n_estimators=500)
xg_ex.fit(X_train, y_train,
eval_set=[(X_train, y_train),
(X_test, y_test)
],
verbose=100
)

[0]	validation_0-logloss:0.63367	validation_1-logloss:0.63390
[100]	validation_0-logloss:0.46775	validation_1-logloss:0.49044
[200]	validation_0-logloss:0.45785	validation_1-logloss:0.48906
[260]	validation_0-logloss:0.45484	validation_1-logloss:0.48893


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7583844691864223, early_stopping_rounds=50,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.0006312831731257507, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.295815862267789, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=7, max_leaves=None,
              min_child_weight=5.333178297222369, missing=nan,
              monotone_constraints=None, n_estimators=500, n_jobs=None,
              num_parallel_tree=None, predictor=None, random_state=None, ...)

In [15]:
xg_ex.score(X_test, y_test)

0.7657458563535912

### 12.3 Defining Parameter Distributions

하이퍼파라미터를 옵션 목록에서 선택하는 것으로 제한하는 대신, 하이퍼옵트 라이브러리에는 값 선택 방법을 정의할 수 있는 다양한 함수가 있습니다.

선택 함수는 범주형 하이퍼파라미터에 대해 불연속적인 값 집합을 지정할 수 있습니다.   
이는 그리드 검색에서 옵션 목록을 열거하는 것과 유사합니다.   

이 함수는 가능한 값의 목록과 이러한 값에 대한 선택적 확률 분포라는 두 가지 인수를 받습니다.

이 함수는 지정된 확률 분포에 따라 목록에서 임의의 값을 반환합니다.

예를 들어 가능한 값 목록 ['a', 'b', 'c']에서 임의의 값을 생성하려면 다음과 같이 코드를 사용하면 됩니다, 다음 코드를 사용할 수 있습니다:

In [16]:
from hyperopt import hp, pyll

In [17]:
pyll.stochastic.sample(hp.choice('value', ['a', 'b', 'c']))

'a'

가능한 값을 확률 분포로 출력하려면

In [18]:
pyll.stochastic.sample(hp.pchoice('value', [(.05, 'a'), (.9, 'b'), (.05, 'c')]))

'b'

- 중요 : 수치형 데이터에 대해서는 choice, pchoice 사용하는 것을 주의 해야 한다.  Hyperopt 라이브러리는 각 값을 순서가 아닌 독립적인 값으로 취급합니다. 검색 알고리즘은 인접한 값의 결과를 활용할 수 없습니다. 
   
Hyperopt를 사용하는 예제에서 다음과 같이 'num_leaves' 및 'subsample'에 대한 검색 공간을 정의할 것을 제안합니다:   

```python
'num_leaves': hp.choice('num_leaves', list(range(20, 250, 10))),
'subsample': hp.choice('subsample', [0.2, 0.4, 0.5, 0.6, 0.7, .8, .9]),
```

그런데 위와 같이 사용하면 안된다. 다음과 같이 설정하여 Hyperopt가 효율적으로 스페이스를 탐색할 수 있도록 해야한다.
```python
'num_leaves': hp.quniform('num_leaves', 20, 250, 10),
'subsample': hp.uniform('subsample', 0.2, .9),
```


uniform 함수는 연속적인 값 범위에 걸쳐 균일한 분포를 지정할 수 있습니다.

이 함수는 uniform 분포의 최소값과 최대값이라는 두 개의 인수를 받고 이 범위 내에서 임의의 부동 소수점 값을 반환합니다.

예를 들어 uniform 함수를 사용하여 0과 1 사이의 임의의 부동 소수점 값을 생성하려면 함수를 사용하여 0과 1 사이의 임의의 부동 소수점 값을 생성하려면 다음 코드를 사용할 수 있습니다:

In [19]:
from hyperopt import hp, pyll

In [20]:
pyll.stochastic.sample(hp.uniform('value', 0, 1))

0.6975627665660606